In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score

import random, os, json
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, GRU, Dropout, Dense
from tensorflow.keras import backend as K
import pickle

import sys
sys.path.append("../../../../libraries/")
import utils

sys.path.append("../../../../classification_architectures/")
import transformer

# Hyperparameters

In [ ]:
#%%
seeds = [20, 30, 45, 70]
results = ""
debug = True
balance = True

epochs = 10000
batch_size = 128
inputShape = 56

lr_scheduler = [0.0001, 0.001, 0.01, 0.1]
dropout = [0.0, 0.1, 0.2, 0.3]

w2 = 0.18
w1 = 0.82

middle_layer_dim = [3, 5, 8, 10, 15, 20, 25, 30, 35, 40, 50]

num_transformer_blocks =  [4, 6]

num_heads = [2, 4, 8]

epsilon = [0.5, 0.9]



hyperparameters = {
    "epochs":epochs,
    "input_shape": inputShape,
    'batch_size': batch_size,
    "w1":w1, "w2":w2, 
    'maskValue':666,
    'patience':30,
    'monitor': 'val_loss', "mindelta": 0,
    'balance': balance, "timeStep": 14,
    'optimizer':'adam',
    'kfold':5,
    'level':3,
    'verbose':0
}

tensor = True
tab = "\t" * hyperparameters["level"]


# Code

In [ ]:
run_model = False

if run_model:
    v_early = []
    loss_dev = []
    v_models = []
    bestHyperparameters_bySplit = {}
    y_pred_by_split = {}

    for i in range(1, 4):
        path = f'../../../../ORIGINAL_DATA/MDR/splits_14_days/majority/split_{str(i)}/'
    
        X_test = np.load(path + f"/X_test_tensor.npy")
        y_test = pd.read_csv(path + f"/y_test.csv", index_col=0)
    
        bestHyperparameters, X_train, y_train, X_val, y_val = transformer.myCVGrid(hyperparameters,
                                                                                   dropout, 
                                                                                   lr_scheduler, 
                                                                                   middle_layer_dim, 
                                                                                   num_transformer_blocks,
                                                                                   num_heads, 
                                                                                   epsilon,
                                                                                   i,                                                              
                                                                                   seeds[i],
                                                                                   path
                                                                                  )
        bestHyperparameters_bySplit[str(i)] = bestHyperparameters
    
        # Save best hyperparameters for current split
        split_directory = './Results_Transformer-CMI/split_' + str(i)
        if not os.path.exists(split_directory):
            os.makedirs(split_directory)
    
        with open(os.path.join(split_directory, f"bestHyperparameters_split_{i}.pkl"), 'wb') as f:
            pickle.dump(bestHyperparameters, f)
    
    
        hyperparameters = {
            "w1":hyperparameters["w1"], "w2":hyperparameters["w2"],                                    
            "timeStep": hyperparameters["timeStep"],
            'epochs':  hyperparameters["epochs"],
            'batch_size': hyperparameters["batch_size"],
            'maskValue': hyperparameters["maskValue"],
            "input_shape": hyperparameters["input_shape"],
            'earlyStopping': True,
            'kfold': hyperparameters["kfold"],
            'monitor': hyperparameters["monitor"],
            "mindelta": hyperparameters["mindelta"],
            "patience": hyperparameters["patience"],
            'balance': hyperparameters["balance"],
            "dropout": bestHyperparameters["dropout"],
            "lr_scheduler": bestHyperparameters["lr_scheduler"],
            "level": 3, 'verbose': 0,

            'middle_layer_dim': bestHyperparameters['middle_layer_dim'],
            'num_transformer_blocks': bestHyperparameters['num_transformer_blocks'],
            'num_heads': bestHyperparameters['num_heads'],
            'epsilon': bestHyperparameters['epsilon'],
        }
    
        #Try on test
        utils.reset_keras()

        model, hist, early = transformer.run_network(
            X_train, y_train.individualMRGerm.values,
            X_val, y_val.individualMRGerm.values,
            hyperparameters, 
            seeds[i]
        )    
    
        v_models.append(model)
        loss_dev.append(hist.history['val_loss'])
    
        y_pred = model.predict(x=[X_test])
        y_pred_by_split[str(i)] = y_pred
        
        with open(os.path.join(split_directory, f"y_pred_split_{i}.pkl"), 'wb') as f:
            pickle.dump(y_pred, f)
    
        # Save model for current split
        model_filename = os.path.join(split_directory, f"model_split_{i}.h5")
        model.save(model_filename)
    
        # Calculate metrics
        metrics_dict = utils.calculate_and_save_metrics(
        y_test.individualMRGerm.values, 
        y_pred, 
        split_directory, 
        split_index=i
        )